## NLP - the Youtube Comment (CN)

In [1]:
from openpyxl import Workbook
wb = Workbook()
wb.save("input/NLP-save1.xlsx")  #Clean-up

In [2]:
import pandas as pd
import numpy as np

import unicodedata
import re

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
data = pd.read_excel('input/NLP-excel.xlsx')
data.head()
#type(data)

,Unnamed: 0,iD,tim,nam,aythimg,mess1,type,channel,Unnamed: 8,The Newest Comment output
0,0,3941548,2022-05-13 14:59:58,Mr. J,https://yt4.ggpht.com/ytc/AKedOLTJ4kuN86d80H6C...,政治防疫 糙,textMessage,ctitv,NaN,14.0
1,1,3941547,2022-05-13 14:59:57,閑聊,https://yt4.ggpht.com/ytc/AKedOLR1B9ZW9uDWo-oG...,3+4及與病毒共存害死那麼多人？死亡人數還不夠多嗎？,textMessage,ctitv,NaN,NaN
2,2,3941546,2022-05-13 14:59:55,Sausan Shunnar,https://yt4.ggpht.com/ytc/AKedOLQbB5RL14GTzuXe...,為什麼報應不在政腐 反而在人民身上 為什麼老百姓要丟性命買單,textMessage,ctitv,NaN,NaN
3,3,3941545,2022-05-13 14:59:54,Heshirley蕃秀,https://yt4.ggpht.com/ytc/AKedOLSCoj6fA5UmDI8Q...,原來政策也可以殺死人,textMessage,ctitv,NaN,NaN
4,4,3941543,2022-05-13 14:59:52,熊男くまお,https://yt4.ggpht.com/ytc/AKedOLREjsH_39Sh1UIt...,"皇明祖訓 {'id': '', 'txt': '', 'url': 'https://www...",textMessage,ctitv,NaN,NaN


In [5]:
data['mess1']

0                                                  政治防疫 糙
1                              3+4及與病毒共存害死那麼多人？死亡人數還不夠多嗎？
2                          為什麼報應不在政腐 反而在人民身上 為什麼老百姓要丟性命買單
3                                              原來政策也可以殺死人
4       皇明祖訓 {'id': '', 'txt': '', 'url': 'https://www...
                              ...                        
2080    茗淨檔以炸片為生{'id': '', 'txt': ':beaming_face_with_...
2081                              中天 在YouTube 真的丟臉 應該在微博報
2082                                林婉儿 小粉紅封城還是喜歡翻牆來看台灣~~
2083                   说全了，应该是今天可筛数字是不是破7W。快筛不够，实际数据没办法获知
2084                      1450多來 衝流量喔，安靜 學習，學海 無崖，少害人了。。。
Name: mess1, Length: 2085, dtype: object

In [6]:
## Excel 把檔案轉換成 dataFrame 轉換成 excel (資料庫匯入用)
import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell.cell import WriteOnlyCell

wb = Workbook()
ws1 = wb.active
ws1.title = "Titles"
for r in dataframe_to_rows(data, index=True, header=True):
    ws1.append(r)

ws1["K1"] = "TheOutput"
ws1["K2"] = str(datetime.date.today())

wb.save("input/NLP-save1.xlsx")
print(wb.sheetnames)

['Titles']


## 1. Words Count : Characters, and Punctuation

In [7]:
# Count the char len
data['char_count'] = data['mess1'].apply(len) 
data['char_count'].head()

# 0     6  政治防疫 糙
# 1    26  3+4及與病毒共存害死那麼多人？死亡人數還不夠多嗎？

0      6
1     26
2     30
3     10
4    218
Name: char_count, dtype: int64

In [8]:
# Count the Sentence Len
data['word_count'] = data['mess1'].apply(lambda x: len(x.split()))
data['word_count'].head() # Word Count(段落)

# 0     2  政治防疫 糙
# 1     1  3+4及與病毒共存害死那麼多人？死亡人數還不夠多嗎？

0     2
1     1
2     3
3     1
4    12
Name: word_count, dtype: int64

In [9]:
# Calculating the Word Density
data['word_density'] = data['char_count'] / (data['word_count']+1)
data['word_density'].head()

0     2.000000
1    13.000000
2     7.500000
3     5.000000
4    16.769231
Name: word_density, dtype: float64

In [10]:
# Calculating the Punctuation Count(, . or and special Punctuation)
import string

punctuation = string.punctuation
data['punctuation_count'] = data['mess1'].apply(
    lambda x:len("".join(_ for _ in x if _ in punctuation))) 
data['punctuation_count'].head()

# 0     0  政治防疫 糙
# 1     1  3+4及與病毒共存害死那麼多人？死亡人數還不夠多嗎？

0     0
1     1
2     0
3     0
4    66
Name: punctuation_count, dtype: int64

## 2.Removing Characters

>Accented characters are important elements which are used to signify emphasis on a particular word during pronunciation or understanding. In some instances, the accent mark also clarifies the meaning of a word, which might be different without the accent. While their use in English is largely limited but there are very good chances that you will come across accented characters/letters in a free text corpus. 

重音字符是重要的元素，用於在發音或理解過程中表示對特定單詞的強調。 在某些情況下，重音標記還可以闡明單詞的含義，如果沒有重音，它可能會有所不同。 雖然它們在英語中的使用在很大程度上受到限制，但您很有可能會在自由文本語料庫中遇到重音字符/字母：Word as résumé, café, prótest, divorcé, coördinate, exposé, latté etc

In [1]:
pip install opencc-python-reimplemented

Note: you may need to restart the kernel to use updated packages.


In [1]:
## 函數方式刪除 Def
from opencc import OpenCC

def drop_numbers(list_text):   #Drop num
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ''.join(list_text_new)

def drop_url(list_text):       #drop_youtube_url{}
    return re.sub(r"\{.+\}","", list_text)

def drop_en(list_text):        #drop_english
    return re.sub(r"[a-zA-Z0-9@+.]","", list_text)

def translate(list_text):
    cc = OpenCC('s2t')
    return cc.convert(list_text)

def regex_change(line):        #reference
    username_regex = re.compile(r"^\d+::")
    url_regex = re.compile(r"""
        (https?://)?
        ([a-zA-Z0-9]+)
        (\.[a-zA-Z0-9]+)
        (\.[a-zA-Z0-9]+)*
        (/[a-zA-Z0-9]+)*""", re.VERBOSE|re.IGNORECASE)
    data_regex = re.compile(u"""年 |月 |日 |(周一) |(周二) | (周三) | (周四) | (周五) | (周六) """, re.VERBOSE) #Drop Date,
    decimal_regex = re.compile(r"[^a-zA-Z]\d+") #Number,
    space_regex = re.compile(r"\s+")     # Space,
    regEx = "[\n”“|,，；;''/?! 。的了是]"  # Dropb: \n、中文符號、|，要刪什麼就寫什麼
    line = username_regex.sub(r"", line)
    line = url_regex.sub(r"", line)
    line = data_regex.sub(r"", line)
    line = decimal_regex.sub(r"", line)
    line = space_regex.sub(r"", line)
    return line

data['mess2'] = data['mess1'].apply(drop_numbers)
data['mess2'] = data['mess2'].apply(drop_url)
data['mess2'] = data['mess2'].apply(drop_en)
#data['mess2'] = data['mess2'].apply(translate)  #run long time
#data['mess2'] = data['mess2'].apply(regex_change)  #reference 自定義
data.head()

ModuleNotFoundError: No module named 'opencc'

In [13]:
## Stopwords & Jieba 解構 方式刪除
from collections import Counter
import jieba
import json

stopword = "input/NLP-stopwords.dat" 
outputword_file = "input/NLP-CN_words.json"

def read_file(file_name):
    fp = open(file_name, "r", encoding="utf-8")
    content_lines = fp.readlines()
    fp.close()
    for i in range(len(content_lines)):
        content_lines[i] = content_lines[i].rstrip("\n")
    return content_lines

def save_file(file_name, content):
    fp = open(file_name, "w", encoding="utf-8")
    fp.write(content)
    fp.close()
    
def delete_stopwords(lines):
    stopwords = read_file(stopword)  #Filter Stopword
    all_words = []
    jieba.load_userdict('input/NLP-userdictCN.txt')   #Filter userdict
    
    # jieba.cut(line, cut_all=True)  全模式 
    # jieba.cut(line, cut_all=False) 精確模式(預設) 
    for line in lines:
        all_words += [word for word in jieba.cut(line) if word not in stopwords]
        #for word in jieba.cut(line):
        #    if word not in stopwords:
        #        print(word)
        
    dict_words = dict(Counter(all_words))
    return dict_words,all_words

In [14]:
# Do the NLP　Action (clean word)
title = []
counts = []
filter_word=[]
sorted_list =[]

#run fun(delete_stopwords) jieba and sorted
jieba_word,dd = delete_stopwords(data['mess2'].tolist()) 
sorted_set = sorted(jieba_word.items(), key=lambda d:d[1], reverse=True)  
#print(sorted_set)   


# convert (Set) to (list) and Sort
for words in sorted_set[:]:  
    wordList = list(words)
    sorted_list += [wordList]
#print(sorted_list)


# Convert (List) To (Table)
for j in range(len(sorted_list)): 
    a = len(sorted_list[j][0])
    if a> 1: 
        title.append(sorted_list[j][0])
        counts.append(sorted_list[j][1])

dicts = {"a":title,"b":counts}
count_data = pd.DataFrame((zip(title,counts)),columns = ['Word','Counts'])
count_data.head(10)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/9q/486czkcn7lv5v0hwbt71twdc0000gn/T/jieba.cache
Loading model cost 0.517 seconds.
Prefix dict has been built successfully.


,Word,Counts
0,台灣,206
1,死亡,193
2,重症,149
3,台湾,124
4,你們,73


In [15]:
from wordcloud import WordCloud

wc = WordCloud(font_path="/System/Library/fonts/PingFang.ttc",width=2000, height=1200)
wc.generate(str(dd))
wc.to_file("input/NLP-wordcloud.jpg")

# https://iter01.com/569982.html

In [16]:
wb.create_sheet("TopWord",1) 
wsM = wb["TopWord"] 

for r in dataframe_to_rows(count_data, index=True, header=True):
    wsM.append(r)

wsM["K1"] = "The Top Word output"
wsM["K2"] = str(datetime.date.today())

wb.save("input/NLP-save1.xlsx")
print(wb.sheetnames)  

['Titles', 'TopWord']


## 3.Text Polarity (情感分析)

>It is the expression that determines the sentimental aspect of an opinion. In textual data, the result of sentiment analysis can be determined for each entity in the sentence, document or sentence. The sentiment polarity can be determined as positive, negative and neutral.

它是決定意見的情感方面的表達方式。 在文本數據中，可以針對句子、文檔或句子中的每個實體確定情感分析的結果。 情緒極性可以確定為正面、負面和中性。

In [17]:
# !pip install textblob

In [18]:
# from textblob import TextBlob

# def get_polarity(text):
#     textblob = TextBlob(str(text.encode('utf-8')))
#     pol = textblob.sentiment.polarity
#     return pol

# data['polarity'] = data['mess1'].apply(get_polarity)
# data['polarity'].head()
# #情感分析不適用於中文

### Text Subjectivity 文本主觀性

>In natural language, subjectivity refers to expression of opinions, evaluations, feelings, and speculations and thus incorporates sentiment. Subjective text is further classified with sentiment or polarity.

在自然語言中，主觀性是指意見、評價、感受和推測的表達，因此包含了情感。 主觀文本進一步分類為情感或極性。

In [19]:
# # Lets calculate the Subjectvity of the Reviews
# def get_subjectivity(text):
#     textblob = TextBlob(str(text.encode('utf-8')))
#     subj = textblob.sentiment.subjectivity
#     return subj

# data['subjectivity'] = data['mess1'].apply(get_subjectivity)
# data['subjectivity'].head()
# #情感分析不適用於中文

### Summarize

In [20]:
# from textblob import TextBlob
# import seaborn as sns
# import matplotlib.pyplot as plt

In [21]:
# data[['polarity','subjectivity']].describe()

In [22]:
# ## Visualizing Polarity and Subjectivity

# plt.subplot(1, 2, 1)
# sns.distplot(data['polarity'])

# plt.subplot(1, 2, 2)
# sns.distplot(data['subjectivity'])

# plt.suptitle('Distribution of Polarity and Subjectivity')
# plt.show()

## NLP

## 留言次數計算

In [23]:
dtc = data.groupby("nam").size().sort_values(ascending=False).reset_index()
dtc.head()

,nam,0
0,网上冲浪李老八,108
1,許立良,98
2,許遵榮,72
3,朱寶寶,45
4,劉沛,45


In [24]:
wb.create_sheet("MostComment",2) 
wsC = wb["MostComment"] 

for r in dataframe_to_rows(dtc, index=True, header=True):
    wsC.append(r)
    
wsC["K1"] = "The Most Comment output"
wsC["K2"] = str(datetime.date.today())

wb.save("input/NLP-save1.xlsx")
print(wb.sheetnames)

['Titles', 'TopWord', 'MostComment']
